<a href="https://colab.research.google.com/github/sharsulkar/H1B_LCA_outcome_prediction/blob/main/notebooks/01_sh_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
%time
data=pd.read_excel('https://www.dol.gov/sites/dolgov/files/ETA/oflc/pdfs/LCA_Disclosure_Data_FY2020_Q1.xlsx')

In [4]:
data.head()

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,ORIGINAL_CERT_DATE,VISA_CLASS,JOB_TITLE,SOC_CODE,SOC_TITLE,FULL_TIME_POSITION,BEGIN_DATE,END_DATE,TOTAL_WORKER_POSITIONS,NEW_EMPLOYMENT,CONTINUED_EMPLOYMENT,CHANGE_PREVIOUS_EMPLOYMENT,NEW_CONCURRENT_EMPLOYMENT,CHANGE_EMPLOYER,AMENDED_PETITION,EMPLOYER_NAME,TRADE_NAME_DBA,EMPLOYER_ADDRESS1,EMPLOYER_ADDRESS2,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_POSTAL_CODE,EMPLOYER_COUNTRY,EMPLOYER_PROVINCE,EMPLOYER_PHONE,EMPLOYER_PHONE_EXT,NAICS_CODE,EMPLOYER_POC_LAST_NAME,EMPLOYER_POC_FIRST_NAME,EMPLOYER_POC_MIDDLE_NAME,EMPLOYER_POC_JOB_TITLE,EMPLOYER_POC_ADDRESS1,EMPLOYER_POC_ADDRESS2,EMPLOYER_POC_CITY,EMPLOYER_POC_STATE,EMPLOYER_POC_POSTAL_CODE,...,AGENT_ATTORNEY_PHONE,AGENT_ATTORNEY_PHONE_EXT,AGENT_ATTORNEY_EMAIL_ADDRESS,LAWFIRM_NAME_BUSINESS_NAME,STATE_OF_HIGHEST_COURT,NAME_OF_HIGHEST_STATE_COURT,WORKSITE_WORKERS,SECONDARY_ENTITY,SECONDARY_ENTITY_BUSINESS_NAME,WORKSITE_ADDRESS1,WORKSITE_ADDRESS2,WORKSITE_CITY,WORKSITE_COUNTY,WORKSITE_STATE,WORKSITE_POSTAL_CODE,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,WAGE_UNIT_OF_PAY,PREVAILING_WAGE,PW_UNIT_OF_PAY,PW_TRACKING_NUMBER,PW_WAGE_LEVEL,PW_OES_YEAR,PW_OTHER_SOURCE,PW_OTHER_YEAR,PW_SURVEY_PUBLISHER,PW_SURVEY_NAME,TOTAL_WORKSITE_LOCATIONS,AGREE_TO_LC_STATEMENT,H-1B_DEPENDENT,WILLFUL_VIOLATOR,SUPPORT_H1B,STATUTORY_BASIS,APPENDIX_A_ATTACHED,PUBLIC_DISCLOSURE,PREPARER_LAST_NAME,PREPARER_FIRST_NAME,PREPARER_MIDDLE_INITIAL,PREPARER_BUSINESS_NAME,PREPARER_EMAIL
0,I-200-19268-393467,Certified,2019-09-25,2019-10-01,NaT,H-1B,"APPLICATION ENGINEER, OMS [15-1199.02]",15-1199,"COMPUTER OCCUPATIONS, ALL OTHER",Y,2019-10-07,2022-10-07,1,1,0,0,0,0,0,"JO-ANN STORES, INC.",JO-ANN FABRIC AND CRAFT STORES,5555 DARROW ROAD,NaN,HUDSON,OH,44236,UNITED STATES OF AMERICA,NaN,3304636787,NaN,451120,KLUSKA,KRYSTA,NaN,"MANAGER, TALENT ACQUISITION",5555 DARROW ROAD,NaN,HUDSON,OH,44236,...,3.122636e+09,NaN,KLO@FRAGOMEN.COM,"FRAGOMEN, DEL REY, BERNSEN & LOEWY, LLP",IL,SUPREME COURT,NaN,N,NaN,5555 Darrow Road,NaN,Hudson,Summit,OH,44224,100000.00,NaN,Year,95118.0,Year,NaN,IV,2018,OES,NaN,NaN,NaN,NaN,Y,N,N,NaN,NaN,NaN,Disclose Business,NaN,NaN,NaN,NaN,NaN
1,I-200-19268-638983,Certified,2019-09-25,2019-10-01,NaT,H-1B,BI DEVELOPER II,15-1132,"SOFTWARE DEVELOPERS, APPLICATIONS",Y,2020-01-08,2023-01-07,1,0,1,0,0,0,0,DENKEN SOLUTIONS INC.,NaN,9170 IRVINE CENTER DRIVE,SUITE # 200,IRVINE,CA,92618,UNITED STATES OF AMERICA,NaN,8017923416,NaN,541512,MADDULA,RAJ,NaN,SOLUTION ARCHITECT,9170 IRVINE CENTER DRIVE,SUITE # 200,IRVINE,CA,92618,...,9.043890e+09,NaN,USCISNOTICES@SCARBOROUGHLAW.COM,"SCARBOROUGH LAW, L.L.C.",FL,FLORIDA SUPREME COURT,NaN,Y,NAVIHEALTH,210 WESTWOOD PL #400,NaN,BRENTWOOD,WILLIAMSON,TN,37027,38.57,38.57,Hour,39.0,Hour,NaN,II,2019,OES,NaN,NaN,NaN,NaN,Y,Y,N,Y,BOTH,NaN,Disclose Business,NaN,NaN,NaN,NaN,NaN
2,I-200-19268-177184,Certified,2019-09-25,2019-10-01,NaT,H-1B,QUALITY ENGINEER,17-2141,MECHANICAL ENGINEERS,Y,2019-10-03,2022-10-02,1,0,0,0,0,1,0,"EPITEC, INC.",NaN,24800 DENSO DRIVE,STE. 150,SOUTHFIELD,MI,48033,UNITED STATES OF AMERICA,NaN,2483536800,NaN,541511,WILDMAN,TAYLOR,NaN,IMMIGRATION LEAD,24800 DENSO DRIVE,STE. 150,SOUTHFIELD,MI,48033,...,2.485200e+09,NaN,NATALIA.SARRAF@ELLISPORTER.COM,"ELLIS PORTER, PLC",MI,SUPREME COURT,NaN,Y,Ford Motor Company,21 Town Center Office (TCO),18900 Michigan Avenue,Dearborn,Wayne,MI,48126,43.50,NaN,Hour,39.0,Hour,NaN,II,2019,OES,NaN,NaN,NaN,NaN,Y,Y,N,Y,BOTH,NaN,Disclose Business,NaN,NaN,NaN,NaN,NaN
3,I-200-19268-936403,Certified,2019-09-25,2019-10-01,NaT,H-1B,"SOFTWARE DEVELOPER, APPLICATIONS",15-1132,"SOFTWARE DEVELOPERS, APPLICATIONS",Y,2019-10-07,2022-10-01,1,0,1,0,0,0,0,"SYSTEMS TECHNOLOGY GROUP, INC.",NaN,"3001 W. BIG BEAVER ROAD, SUITE 500",NaN,TROY,MI,48084,UNITED STATES OF AMERICA,NaN,2486439010,NaN,541511,ARVAL,SREESHA,NaN,IMMIGRATION PARALEGAL,"3001 W. BIG BEAVER ROAD, SUITE 500",NaN,TROY,MI,48084,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Ford Motor Company,21001 Van Born Road,NaN,Taylor,Wayne,MI,48180,57.69,57.69,Hour,53.0,Hour,N

In [5]:
data_copy=data.copy()

In [9]:
#Check for Target value's distribution over the data
print(data.groupby(['CASE_STATUS']).size())

CASE_STATUS
Certified                104216
Certified - Withdrawn      5261
Denied                      728
Withdrawn                  1812
dtype: int64


In [ ]:
df_data_statistics=pd.DataFrame(data=None,index=data.columns,columns=['Dtype','%missing','unique values'])

In [ ]:
#write reusable function to display
#record count per class
print(data.groupby(['CASE_STATUS']).size())
#df shape - records and features
print(df.shape())



In [ ]:
#dataframe statistics
for column in data.columns:
  #identify numeric, non-numeric and date columns
  df_data_statistics.Dtype.loc[column]=data[column].Dtype

#% missing data for each column
#Cardinality of each column


In [21]:
data.dtypes

CASE_NUMBER                        object
CASE_STATUS                        object
RECEIVED_DATE              datetime64[ns]
DECISION_DATE              datetime64[ns]
ORIGINAL_CERT_DATE         datetime64[ns]
                                ...      
PREPARER_LAST_NAME                 object
PREPARER_FIRST_NAME                object
PREPARER_MIDDLE_INITIAL            object
PREPARER_BUSINESS_NAME             object
PREPARER_EMAIL                     object
Length: 96, dtype: object